<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Submit an Existing Notebook to AzureML
## Introduction to Azure Machine Learning  
The **[Azure Machine Learning service (AzureML)](https://docs.microsoft.com/azure/machine-learning/service/overview-what-is-azure-ml)** provides a cloud-based environment you can use to prep data, train, test, deploy, manage, and track machine learning models. By using Azure Machine Learning service, you can start training on your local machine and then scale out to the cloud. With many available compute targets, like [Azure Machine Learning Compute](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) and [Azure Databricks](https://docs.microsoft.com/en-us/azure/azure-databricks/what-is-azure-databricks), and with [advanced hyperparameter tuning services](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters), you can build better models faster by using the power of the cloud.

Data scientists and AI developers use the main [Azure Machine Learning Python SDK](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py) to build and run machine learning workflows with the Azure Machine Learning service. You can interact with the service in any Python environment, including Jupyter Notebooks or your favorite Python IDE. The Azure Machine Learning SDK allows you the choice of using local or cloud compute resources, while managing and maintaining the complete data science workflow from the cloud.
![AzureML Workflow](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/overview-what-is-azure-ml/aml.png)

This noteboook provides a scaffold to directly submit an existing notebook to AzureML compute targets. Now a user doesn't have to rewrite the training script, instead, just by replacing file name, now user can submit notebook directly.

### Advantages of using AzureML:
- Manage cloud resources for monitoring, logging, and organizing your machine learning experiments.
- Train models either locally or by using cloud resources, including GPU-accelerated model training.
- Easy to scale out when dataset grows - by just creating and pointing to new compute target

### Prerequisities
   - **Azure Subscription**
     - If you don’t have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning service today](https://azure.microsoft.com/en-us/free/services/machine-learning/).
     - You get credits to spend on Azure services, which will easily cover the cost of running this example notebook. After they're used up, you can keep the account and use [free Azure services](https://azure.microsoft.com/en-us/free/). Your credit card is never charged unless you explicitly change your settings and ask to be charged. Or [activate MSDN subscriber benefits](https://azure.microsoft.com/en-us/pricing/member-offers/credit-for-visual-studio-subscribers/), which give you credits every month that you can use for paid Azure services.

# Setup environment

### Install azure.contrib.notebook package
We need notebook_run_config from azureml.contrib.notebook to run this notebook. Since azureml.contrib.notebook contains experimental components, it's not included in generated .yaml file by default.

In [3]:
pip install "azureml.contrib.notebook>=1.0.21.1"

     |████████████████████████████████| 312 kB 11 kB/s eta 0:00:01
     |████████████████████████████████| 455 kB 17 kB/s eta 0:00:016
     |████████████████████████████████| 2.2 MB 24 kB/s eta 0:00:019
     |████████████████████████████████| 40 kB 18 kB/s eta 0:00:014
     |████████████████████████████████| 1.3 MB 15 kB/s eta 0:00:01
     |████████████████████████████████| 147 kB 21 kB/s eta 0:00:01
     |████████████████████████████████| 547 kB 18 kB/s eta 0:00:01
     |████████████████████████████████| 94 kB 21 kB/s eta 0:00:01
     |████████████████████████████████| 141 kB 14 kB/s eta 0:00:01
     |████████████████████████████████| 312 kB 18 kB/s eta 0:00:01
     |████████████████████████████████| 55 kB 22 kB/s eta 0:00:014
     |████████████████████████████████| 796 kB 18 kB/s eta 0:00:01
     |████████████████████████████████| 85 kB 18 kB/s eta 0:00:01


     |████████████████████████████████| 52 kB 16 kB/s eta 0:00:013
     |████████████████████████████████| 54 kB 21 kB/s eta 0:00:014
     |████████████████████████████████| 77 kB 11 kB/s  eta 0:00:01
     |████████████████████████████████| 45 kB 20 kB/s eta 0:00:014
     |████████████████████████████████| 165 kB 23 kB/s eta 0:00:01
     |████████████████████████████████| 146 kB 18 kB/s eta 0:00:01
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.0.7
    Uninstalling nbconvert-6.0.7:
      Successfully uninstalled nbconvert-6.0.7
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >

In [5]:
pip install "ruamel"

ERROR: Could not find a version that satisfies the requirement ruamel (from versions: none)
ERROR: No matching distribution found for ruamel
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import azureml.core
from azureml.core import Workspace
from os import path, makedirs
import shutil
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.core.runconfig import RunConfiguration
from azureml.contrib.notebook import NotebookRunConfig, PapermillExecutionHandler
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies
from tempfile import TemporaryDirectory

print("azureml.core version: {}".format(azureml.core.VERSION))

ModuleNotFoundError: No module named 'ruamel'

### Connect to an AzureML workspace

An [AzureML Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows. In particular, an Azure ML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, deployment, inferencing, and the monitoring of deployed models.

The function below will get or create an AzureML Workspace and save the configuration to `aml_config/config.json`.

It defaults to use provided input parameters or environment variables for the Workspace configuration values. Otherwise, it will use an existing configuration file (either at `./aml_config/config.json` or a path specified by the config_path parameter).

Lastly, if the workspace does not exist, one will be created for you. See [this tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace#portal) to locate information such as subscription id.

In [3]:
ws = Workspace.create(
    name="<WORKSPACE_NAME>",
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    location="<WORKSPACE_REGION>"
    exist_ok=True,
)

This example uses run-based AzureML Compute, which requires minimum setup. Alternatively, you can use persistent compute target (cpu or gpu cluster) or remote virtual machines. For more details, see [How to set up training targets](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets).

Cost-wise, according to Azure [Pricing calculator](https://azure.microsoft.com/en-us/pricing/calculator/), with example VM size `STANDARD_D2_V2`, it costs a few dollars to run this notebook, which is well covered by Azure new subscription credit. For billing and pricing questions, please contact [Azure support](https://azure.microsoft.com/en-us/support/options/).

**Note**:
- See list of all available VM sizes [here](https://docs.microsoft.com/en-us/azure/templates/Microsoft.Compute/2018-10-01/virtualMachines?toc=%2Fen-us%2Fazure%2Fazure-resource-manager%2Ftoc.json&bc=%2Fen-us%2Fazure%2Fbread%2Ftoc.json#hardwareprofile-object).
- As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [these instructions](https://docs.microsoft.com/en-us/azure/azure-supportability/resource-manager-core-quotas-request) on the default limits and how to request more quota.



### Create or Attach Azure Machine Learning Compute 

We create a cpu cluster as our **remote compute target**. If a cluster with the same name already exists in your workspace, the script will load it instead. You can read [Set up compute targets for model training](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets) to learn more about setting up compute target on different locations. You can also create GPU machines when larger machines are necessary to train the model.

According to Azure [Pricing calculator](https://azure.microsoft.com/en-us/pricing/calculator/), with example VM size `STANDARD_D2_V2`, it costs a few dollars to run this notebook, which is well covered by Azure new subscription credit. For billing and pricing questions, please contact [Azure support](https://azure.microsoft.com/en-us/support/options/).

**Note**:
- 10m and 20m dataset requires more capacity than `STANDARD_D2_V2`, such as `STANDARD_NC6` or `STANDARD_NC12`. See list of all available VM sizes [here](https://docs.microsoft.com/en-us/azure/templates/Microsoft.Compute/2018-10-01/virtualMachines?toc=%2Fen-us%2Fazure%2Fazure-resource-manager%2Ftoc.json&bc=%2Fen-us%2Fazure%2Fbread%2Ftoc.json#hardwareprofile-object).
- As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [these instructions](https://docs.microsoft.com/en-us/azure/azure-supportability/resource-manager-core-quotas-request) on the default limits and how to request more quota.
---
#### Learn more about Azure Machine Learning Compute
<details>
    <summary>Click to learn more about compute types</summary>
    
[Azure Machine Learning Compute](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created within your workspace region and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user.

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service.

You can provision a persistent AmlCompute resource by simply defining two parameters thanks to smart defaults. By default it autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

In addition to vm_size and max_nodes, you can specify:
- **min_nodes**: Minimum nodes (default 0 nodes) to downscale to while running a job on AmlCompute
- **vm_priority**: Choose between 'dedicated' (default) and 'lowpriority' VMs when provisioning AmlCompute. Low Priority VMs use Azure's excess capacity and are thus cheaper but risk your run being pre-empted
- **idle_seconds_before_scaledown**: Idle time (default 120 seconds) to wait after run completion before auto-scaling to min_nodes
- **vnet_resourcegroup_name**: Resource group of the existing VNet within which AmlCompute should be provisioned
- **vnet_name**: Name of VNet
- **subnet_name**: Name of SubNet within the VNet
</details>
---

In [4]:
# Remote compute (cluster) configuration. If you want to save the cost more, set these to small.
VM_SIZE = 'STANDARD_D2_V2'
# Cluster nodes
MIN_NODES = 0
MAX_NODES = 2

CLUSTER_NAME = 'cpucluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=CLUSTER_NAME)
    print("Found existing compute target")
except:
    print("Creating a new compute target...")
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(
        vm_size=VM_SIZE,
        min_nodes=MIN_NODES,
        max_nodes=MAX_NODES
    )
    # Create the cluster with the specified name and configuration
    compute_target = ComputeTarget.create(ws, CLUSTER_NAME, compute_config)
    # Wait for the cluster to complete, show the output log
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target


This example uses persistent compute target, which requires creation of the computing resource for first time. Alternatively, you can use run-based compute target or remote virtual machines.

In [5]:
NOTEBOOK_NAME = 'sar_movielens.ipynb' # use different notebook file name if trying to run other notebooks
experiment_name = NOTEBOOK_NAME.strip(".ipynb")
tmp_dir = TemporaryDirectory()
notebook_path = path.join(tmp_dir.name, NOTEBOOK_NAME)
source_path = path.join('00_quick_start/', NOTEBOOK_NAME)
shutil.copy(source_path, notebook_path)
        
exp = Experiment(workspace=ws, name=experiment_name)

run_config = RunConfiguration()
run_config.target = "cpucluster" # possible to use alternative compute targets
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.user_managed_dependencies = False
run_config.auto_prepare_environment = True
run_config.environment.python.conda_dependencies = CondaDependencies.create(python_version='3.6.2')
run_config.environment.python.conda_dependencies.add_pip_package('sklearn')

cfg = NotebookRunConfig(source_directory='../',
                            notebook='notebooks/00_quick_start/' + NOTEBOOK_NAME,
                            output_notebook='outputs/out.ipynb',
                            parameters={"MOVIELENS_DATA_SIZE": "100k", "TOP_K": 10},
                            run_config=run_config)


`exp.submit()` will submit source_directory folder and designate notebook to run

In [ ]:
run = exp.submit(cfg)
run

Above cell should output similar table as below
![Experiment submit output](https://recodatasets.z20.web.core.windows.net/images/aml_papermill_cell_output.jpg)
After clicking "Link to Azure Portal", experiment run details tab looks like this
![Azure Portal Experiment](https://recodatasets.z20.web.core.windows.net/images/aml_papermill_portal.jpg)
You can find the executed notebook out.ipynb under outputs tab.
![Output file](https://recodatasets.z20.web.core.windows.net/images/aml_papermill_outputs_tab.jpg)
#### Jupyter widget

You can use a Jupyter widget to monitor the progress of the run. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [7]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

You can view logged metrics with run.get_metrics()

In [11]:
# run below after run is complete, otherwise metrics is empty
metrics = run.get_metrics()
print(metrics)

{'MOVIELENS_DATA_SIZE': '100k', 'TOP_K': 10, 'map': 0.10485162234730652, 'ndcg': 0.3704872048540983, 'precision': 0.3211252653927813, 'recall': 0.17416985459670545, 'test_time': 0.15025854110717773, 'train_time': 0.29204869270324707}


# Deprovision compute resource
To avoid unnecessary charges, if you created compute target that doesn't scale down to 0, make sure the compute target is deprovisioned after use.

In [9]:
# delete () is used to deprovision and delete the AmlCompute target. 
# do not run below before experiment completes

# compute_target.delete()

# deletion will take a few minutes. You can check progress in Azure Portal / Computing tab

In [10]:
# clean up temporary directory
tmp_dir.cleanup()